# Tworzenie zasobów

Algorytmy wykorzystywane w problemach przetwarzania języka naturalnego opierają najczęściej swoje działanie o analizę dużych korpusów danych. O ile w zadaniach konkursowych często odpowiednie dane są już przygotowane, o tyle tworząc własne eksperymenty, często musimy sami pozyskać dane i przetransformować do użytecznej postaci.

Dzisiejsze laboratoria dotyczyć będą tworzenia korpusów danych.

## Automatyczne pozyskiwanie surowych danych tekstowych
Dotychczas omawiane metody działały na surowym tekście, który transformowany był do odpowiedniej reprezentacji wektorowej (Bag of words, bag of ngrams, embeddingi). Jak zautomatyzować pozyskiwanie takich surowych danych z internetu?

W tej części skupimy się na stworzeniu automatycznego pobieracza danych, który działać będzie w dwóch "obszarach":
<ol>
<li>crawler: moduł odwiedzający kolejne strony internetowy</li>
<li>scraper: moduł ekstrahujący treść z konkretnych stron internetowych</li>
</ol>

Wykorzystajmy do tego dwie biblioteki: 

**urllib** - do odwiedzania stron

**BeautifulSoup** - do parsowania danych (np. w formacie HTML).

## Zadanie1: Napisz prosty ekstraktor danych ze stron WWW odwiedzający kilka podstron
Ekstraktor ma odwiedzić zadaną stronę internetową, pobrać zawartość wszystkich tekstów wewnątrz paragrafów (wewnątrz tagów P zawartych w pobranym dokumencie HTML), a następnie odwiedzić 5 dowolnych linków z tej strony i z nich analogicznie pobrać zawartość.
Łącznie powinniśmy otrzymać dane z 6 adresów internetowch (strona główna + 5 linków ze strony głównej).

Do napisania crawlera przydać się mogą następujące funkcje:

urllib.request.urlopen() - do pobrania zawartości strony
findAll() na obiekcie BeautifulSoup, można ją wykorzystać do przeiterowania po wszystkich tagach danego rodzaju
get_text() - Istnieje duża szansa, że wewnątrz tagów P znajdą się również inne tagi HTML, chcielibyśmy oczyścić 
z nich tekst. Można to zrobić albo z wyrażeniami regularnymi (robiliśmy takie zadanie na pierwszych laboratoriach!), albo użyć właśnie funkcji get_text() z BeautifulSoup

Linki do dokumentacji:
urllib, pobieranie danych: https://docs.python.org/3/howto/urllib2.html
beautifulSoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/ (przeczytanie QuickStart jest wystarczające do zrobienia tego zadania)


In [2]:
import urllib.request
from bs4 import BeautifulSoup
from tqdm import tqdm

URL = "http://www.cs.put.poznan.pl/alawrynowicz/pjn/"

# wziete ze stackoverflow (aby nie instalowac do tego pakietu)
def is_url(url) -> bool:
    import re
    regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return re.match(regex, str(url)) is not None

#== SCRAPER
def get(url) -> (list, list):
    all_p, all_a = [], []
    response = urllib.request.urlopen(url)
    raw_html = response.read()
    soup = BeautifulSoup(raw_html)
    # print(f"\033[92m{url}\033[0m\n")
    for p in tqdm(soup.find_all('p')):
        p_cur = p.get_text().strip()
        if p_cur == "": continue
        all_p.append(p_cur)
        # print("==>", p_cur, "\n")
    for a in soup.find_all('a'):
        href = a.get('href')
        if is_url(href):
            # print(a.get('href'))
            all_a.append(href)
    return all_p, all_a
    
#== CRAWLER 
all_p, all_a = get(URL)
for url in all_a[0:5]:
    _all_p, _ = get(url)
    all_p += _all_p

#== RESULT
# FIXME: zgodnie ze specyfikacja "cale tagi <p>",
#        wiec nie wydzielam pojedynczych zdan
print(all_p)

100%|██████████| 6/6 [00:00<00:00, 8499.10it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 11/11 [00:00<00:00, 4347.66it/s]

['Przetwarzanie Języka Naturalnego', 'Strona poświęcona przedmiotowi obieralnemu.', 'Przedmiot poświęcony jest zagadnieniom przetwarzania języka naturalnego.\nSzczegółowe tematy obejmują podstawowe techniki przetwarzania tekstu (wyrażenia regularne, filtrowanie wyrazów funkcyjnych, segmentacja, lematyzacja, odległość edycyjna), klasyfikację tekstu, analizę gramatyczną, statystyczne modelowanie języka, wektorowe reprezentacje dystrybucyjne (embeddings), reprezentację wiedzy, ekstrakcję informacji i wiedzy z danych tekstowych, zagadnienia uczenia głębokiego w przetwarzaniu języka naturalnego oraz zastosowania do celów analizy sentymentu i opinii oraz w systemach dialogowych.', 'Slajdy z wykładów oraz notatniki z ćwiczeniami laboratoryjnymi znajdują się na uczelnianej platformie eLearningowej PUT LMS Moodle.\nDla celów przedmiotu stworzone zostało również forum dyskusyjne Piazza, które można wykorzystać w celu kontaktowania się z prowadzącymi i zadawania pytań dotyczących przedmiotu. Foru

# Zadanie 2 - CONLL
Dane ustrukturyzowane w formacie CONLL.

Niektóre algorytmy korzystają z dodatkowych metadanych opisujących poszczególne tokeny (słowa). Bardzo popularnym formatem zapisu takich danych jest format CONLL. 

Reprezentacja CONLL polega na tym, że dany tekst dzielony jest na zdania, a następnie każde zdanie dzielone jest na tokeny (tokenizowane). Następnie dla każdego tokenu tworzymy listę opisującą cechy tego tokenu (słowa).
Poniżej przykład wektora opisującego każdy token zadanego tekstu:
<ol>
    <li>ID - numer porządkowy tokenu w zdaniu</li>
    <li>text - tekst tokenu w formie nieprzetworzonej</li>
    <li>Part of Speech tag (POS tag) - informacja o części mowy, która powiązana jest z tym słowem </li>
    <li>is digit - flaga (o wartościach 0 lub 1), która informuje nas czy dany token jest liczbą</li>
    <li>is punct - flaga (o wartościach 0 lub 1), która informuje nas czy dany token jest znakiem interpunkcyjnym</li>
</ol>

Wektory cech dla kolejnych słów zapisywane są pod sobą. **Separatorem cech w wektorze jest pojedyncza spacja.**

**Zdania zwyczajowo oddzielamy od siebie podwójnym znakiem nowej linii.**

Historycznie CONLL był bardzo konkretnym formatem danych w którym mieliśmy z góry narzucone cechy (np. format CONLL-U https://universaldependencies.org/docs/format.html). Liczba cech ewoluowała jednak w czasie i w wielu miejscach CONLL stał się synonimem ogólnego formatu, w którym dobór cech zależy tylko od nas, jednak stałym jest zapis sekwencji tokenów jako sekwencji wierszy w tekście, gdzie każdy wiersz jest listą oddzielonych spacją wartości (cech), a zdania oddzielone są od siebie podwójnym znakiem nowej linii.


### Przykład:

Tekst: Kasia kupiła 2 lizaki: truskawkowy i zielony. Kasia używa Apple IPhone 5 i IPad.

Reprezentacja CONLL **(spacje separujące kolumny zostały zwielokrotnione na potrzeby zwiększenia czytelności)**
<pre>
1 Kasia  RZECZOWNIK 0 0
2 kupiła CZASOWNIK  0 0
3 2      LICZEBNIK  1 0
4 lizaki RZECZOWNIK 0 0
5 .      _          0 1

1 Kasia  RZECZOWNIK 0 0
2 używa  CZASOWNIK  0 0
3 Apple  RZECZOWNIK 0 0
4 IPhone RZECZOWNIK 0 0
5 5      LICZEBNIK  1 0
6 i      SPÓJNIK    0 0
7 iPad   RZECZOWNIK 0 0
8 .      _          0 1
</pre>

**Zadanie**: Napisz funkcję, która z zadanego tekstu w formie surowego tekstu stworzy reprezentację CONLL opisaną wcześniej wymienionymi atrybutami (ID, text, POS-tag, is_digit, is_punct).

Wykorzystaj sentence splitter i tokenizator z NLTK. Do uzyskania informacji o POS-tagach każdego tokenu wykorzystaj funkcję nltk.pos_tag(). W kolumnie związanej z POS-tagiem zapisz pos tag w takiej formie, w jakiej uzyskamy go z funkcji pos_tag (pos_tag() zwraca formy skrótowe, np. 'NN' dla rzeczowników), nie trzeba więc zamieniać napisu "NN" na "RZECZOWNIK".


In [18]:
import string
from nltk import sent_tokenize, pos_tag
from nltk.tokenize import TweetTokenizer
from nltk import text_type
tknzr = TweetTokenizer() # dobry dla nowych domen

# FIXME: nigdzie nie pisze ze mam to zwrocic?
#        wiec rozumiem ze ktos chce stdout > plik.conll
def generate_conll(text) -> None:
    for sent in sent_tokenize(text):
        print("")
        tokens = tknzr.tokenize(sent)
        pos_tokens = pos_tag(tokens)
        i = 1
        for pos_token in pos_tokens:
            b1 = int(text_type.isdigit(pos_token[0]))
            b2 = int(pos_token[0] in string.punctuation)
            print(i, pos_token[0], pos_token[1], b1, b2)
            i += 1


generate_conll("Kate uses IPhone 55 and IPad. Kate bought 2 lolipops.")


1 Kate NNP 0 0
2 uses VBZ 0 0
3 IPhone NNP 0 0
4 55 CD 1 0
5 and CC 0 0
6 IPad NNP 0 0
7 . . 0 1

1 Kate NNP 0 0
2 bought VBD 0 0
3 2 CD 1 0
4 lolipops NNS 0 0
5 . . 0 1



Wyobraźmy sobie teraz, że chcielibyśmy wykrywać wzmianki o urządzeniach elektronicznych w tekście. W jaki sposób zakodować informację o (potencjalnie wielotokenowych) nazwach produktów w CONLL, tak, aby później móc wykonać proces uczenia?

Dodajmy w naszym CONLLu dodatkową kolumnę reprezentującą informację o urządzeniach elektronicznych.
Nazwy urządzeń mogą składać się potencjalnie z wielu słów.
Do zakodowania wielotokenowych tekstów używa się najczęściej notacji IOB, gdzie każda literka skrótu oznacza interpretację aktualnego słowa:
<ul>
    <li> B = begin, marker, który mówi, że aktualne słowo to początek nazwy </li>
    <li> I = inside, marker, który mówi, że aktualne słowo to kontynacja nazwy, która rozpoczyna się wystąpieniem wcześniejszego B</li>
    <li> O = outside, marker, który mówi, że aktualne słowo nie jest interesującą nas nazwą (jest poza nią) </li>
</ul>

Po dodaniu nowej kolumny (na końcu) nasz CONLL przybiera postać:

<pre>
1 Kasia  RZECZOWNIK 0 0 O
2 kupiła CZASOWNIK  0 0 O
3 2                 1 0 O
4 lizaki RZECZOWNIK 0 0 O
5 .      _          0 1 O

1 Kasia  RZECZOWNIK 0 0 O
2 używa             0 0 O
3 Apple  RZECZOWNIK 0 0 B
4 IPhone RZECZOWNIK 0 0 I
5 5                 1 0 I
6 i      SPÓJNIK    0 0 O
7 iPad   RZECZOWNIK 0 0 B
8 .      _          0 1 0
</pre>

Zwróćcie Państwo uwagę na ostatnią kolumnę, czytając tekst od góry w dół, wystąpienie literki "B" oznacza początek interesującej frazy (Apple), jeśli zaraz za "B" pojawia się sekwencja oznaczona jako "I" - kolejne tokeny stanowią kontynuację interesującej nas frazy, w tym przypadku 3 tokeny "Apple IPhone 5" tworzą jeden byt. Poza tym widzimy, że "iPad" stanowi osobny, jednotokenowy byt.

Po co rozróżniać pomiędzy "B", "I" i "O", czy nie można uwzględnić tylko dwóch tagów "wewnątrz frazy", "poza frazą"? Teoretycznie można, ale wprowadzimy w ten sposób sytuacje niejednoznaczne. 

Sprawdźmy to na przykładzie sekwencji "XBox Playstation" reprezentującej 2 osobne byty. Używając tagowania IOB nasza sekwencja wyglądałaby tak:

XBox B
PlayStation B

Widzimy więc, że dwa tagi "B" oznaczają dwa początki osobnych fraz. Co jednak gdybyśmy używali tagów "wewnątrz (interesującej nas) frazy", "poza (interesującą nas) frazą"?

XBox "wewnątrz (interesującej nas) frazy"
Playstation "wewnątrz (interesującej nas) frazy"

W tej sytuacji oznaczyliśmy poprawnie oba tokeny jako części interesujących nas fraz. Jednak nie wiemy, czy XBox Playstation to jedna, czy dwie osobne frazy (byty) -- stąd format IOB jest zdecydowanie bezpieczniejszym wyborem.

**Zadanie**: Napisz funkcję, która wygeneruje CONLL z uwzględnieniem tagów IOB dotyczących urządzeń.
Nasza funkcja posiada teraz dodatkowy argument devices, który zawiera listę obiektów, które opisują gdzie (przesunięcie znakowe) znajduje się początek i koniec wzmianek.


In [44]:
import string
from nltk import sent_tokenize, pos_tag
from nltk.tokenize import TweetTokenizer
from nltk import text_type
from pprint import pprint
from copy import copy
tknzr = TweetTokenizer()

def generate_CONLL(text, devices=[]) -> None:
    # zrobione jest tak aby w wyniku tokenizacji,
    # nie przesunely sie wyrazy miedzy soba i nie zaznaczyc
    # przez to innych
    print("=== DEBUG ===")
    pprint(devices)
    text_mask = copy(text)
    for device in devices:
        masked = text_mask[device["begin"]:device["end"]]\
                 .translate({ord(c):'X' for c in string.printable.replace(" ", "")})
        text_mask = text_mask[0:device["begin"]] + masked + text_mask[device["end"]:]
    print("@1: ", text)
    print("@2: ", text_mask)
    print("=== DEBUG ===")
    sents = zip(sent_tokenize(text), sent_tokenize(text_mask))
    for sent, sent_mask in sents:
        print("")
        tokens = tknzr.tokenize(sent)
        tokens_mask = tknzr.tokenize(sent_mask)
        pos_tokens = pos_tag(tokens)
        pos_tokens_mask = pos_tag(tokens_mask)
        i = 1
        last_unicorn = False
        for pos_token, pos_token_mask in zip(pos_tokens, pos_tokens_mask):
            b1 = int(text_type.isdigit(pos_token[0]))
            b2 = int(pos_token[0] in string.punctuation)
            _unicorn_char = list(set(pos_token_mask[0]))
            is_unicorn = _unicorn_char[0] == "X" and len(_unicorn_char) == 1
            if is_unicorn and not last_unicorn:
                b3 = "B"
            elif is_unicorn and last_unicorn:
                b3 = "I"
            else:
                b3 = "O"
            print(i, pos_token[0], pos_token[1], b1, b2, b3)
            last_unicorn = is_unicorn
            i += 1

# parametr devices to lista słowników w którym mamy informację o numerze znaku na którym fraza się zaczyna i kończy (zobacz: próba wywołania w ostatniej linijce) (litera I z Iphone występuje na 10 znaku)
# Zapoznaj się z dokumentacją SpaCy (obiekt Token), aby zobaczyć jak wydobyć informację o pozycji danego słowa w zdaniu/dokumencie.
    
generate_CONLL("Kate uses IPhone 5 and IPad. Kate bought 2 lolipops.", devices=[{"begin": 10, "end":18}, {"begin": 23, "end": 27}])

=== DEBUG ===
[{'begin': 10, 'end': 18}, {'begin': 23, 'end': 27}]
@1:  Kate uses IPhone 5 and IPad. Kate bought 2 lolipops.
@2:  Kate uses XXXXXX X and XXXX. Kate bought 2 lolipops.
=== DEBUG ===

1 Kate NNP 0 0 O
2 uses VBZ 0 0 O
3 IPhone NNP 0 0 B
4 5 CD 1 0 I
5 and CC 0 0 O
6 IPad NNP 0 0 B
7 . . 0 1 O

1 Kate NNP 0 0 O
2 bought VBD 0 0 O
3 2 CD 1 0 O
4 lolipops NNS 0 0 O
5 . . 0 1 O


Często chcemy w tekście naraz oznaczać byty, które należą do różnych kategorii, np. lokacje, numery telefonów, daty, wzmianki o osobach. W takich sytuacjach używa się również kodowania IOB jednak wzbogaca się etykiety o odpowiednie informacje używając formatu:

{tag IOB}-{etykieta kategorii}

Stąd daty przyjmują oznaczenia: B-DATE / I-DATE, osoby B-PERSON / I-PERSON, numery telefonów B-PHONENUMBER / I-PHONENUMBER, lokacje: B-LOCATION / I-LOCATION itp. Wiemy zatem czy dany token należy do interesującej nas frazy i do jakiej kategorii przypisana jest ta fraza.